# Outline
1. Background
1. Load and Process
1. Algorithm 3.5
    1. Number of words and emojis
    1. Number of emojis
    1. Number of words
    1. Save results
    1. Number of unique words (WIP)
1. EDA
    1. Words + emojis
    1. Emojis
    1. Words

# Background

A modification of Algorithm 3 looking at the number of words per each 2 min chunk

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

from helpers import data_handler as d
from helpers import graph_helpers as gr

# Load and Process

In [2]:
data = json.load(open('data/big_data.json'))

In [3]:
big_df = d.organize_twitch_chat(data)

# Algorithm 3.5

## Number of words + emoticons

In [7]:
def results_formatter(dataframe, goal):
    '''
    Creates a new df `results` that contains the total number of words in the dataframe, the time
    the time the dataframe started and ended
    input
    -----
    dataframe: pd.DataFrame
        Dataframe with all the hours and chunks labeled, and num_words calculated
    goal: str
        Col name that has calculated results (ex: num_words, chat_rate, etc.)
    output
    ------
    results: pd.DataFrame
        Dataframe with `hour`, `chunk`, `start`, `end`, `num_words` columns
    '''
    hour_list = []
    chunk_list = []
    time_start_list = []
    time_end_list = []
    goal_list = []
    
    dataframe = dataframe.sort_values('created_at')
    # label each chunk with unique ID
    dataframe['category'] = dataframe['hour'].astype(str) + dataframe['chunk'].astype(str)
    
    for cat in dataframe['category'].unique():
        temp_df = dataframe[dataframe['category'] == cat] 
        hour_list.append(temp_df.iloc[0, -3])  # assumes hour col is 3 from end
        chunk_list.append(temp_df.iloc[0, -2])  # assumes hunk col is 2 from end
        time_start_list.append(temp_df.iloc[0, 0])  # assume created_at col is first
        time_end_list.append(temp_df.iloc[-1, 0])  # assume created_at col is first
        goal_list.append(temp_df[goal].sum())

    results = pd.DataFrame({
        'hour':hour_list,
        'chunk':chunk_list,
        'start':time_start_list,
        'end':time_end_list,
        goal:goal_list
    })
    return results

In [8]:
def num_words_in_chat(dataframe):
    '''
    Creates a new col `num_words_emo` that contains the number of words in each message
    '''
    word_bag = []
    body_split = dataframe['body'].str.split(' ')  # split each string at the whitespace
    for row in body_split:
        num_words = len(row)
        word_bag.append(num_words)
    
    dataframe['num_words_emo'] = word_bag
    return dataframe

## Number of emojis

In [9]:
big_df.iloc[2402]['body']

'lii1more lii1more lii1more'

In [10]:
big_df.iloc[2402]['emoticons']

[{'_id': '303368959', 'begin': 0, 'end': 7},
 {'_id': '303368959', 'begin': 9, 'end': 16},
 {'_id': '303368959', 'begin': 18, 'end': 25}]

Emoticon column can be seen as a "marker" for which chat message contained one or more emoticon. Each "marker" (row with emoticon in it) contains a list of dictionaries with:
* `_id`: twitch id of that emoticon
* `begin`: character index of the start of the emoticon reference
* `end`: character index of the end of the emoticon reference

1 dictionary = 1 emoticon

__This cell is the "algorithm"__

In [15]:
# find the number words (this includes emoticons)
big_df = num_words_in_chat(big_df)
# find the number of emoticons used in each message
# if the cell has a list, it has emoticon(s)
big_df['num_emo'] = big_df['emoticons'].apply(lambda x: len(x) if type(x) == list else 0)
# find the number of words only
big_df['num_words_only'] = big_df['num_words_emo'] - big_df['num_emo']

__The following function is just formatting__

In [23]:
def chunker(big_df, min_):
    '''
    Runs big_df through classes and functions to split it into hours/chunks
    '''
    # split into hours
    dfs = d.dfSplitter(big_df)
    dfs.find_rest()

    hour_list = dfs.result
    first_stamp = hour_list[0]
    del hour_list[0]
    
    big_unique = big_df['_id'].unique()
    
    # split into minutes
    chunk_list = []
    for i in range(len(hour_list)):
        hour = hour_list[i]
        xmc = d.xminChats(hour, big_unique, min_=min_)
        xmc.find_rest()
        for chunk_num in range(len(xmc.result)):
            chunk = xmc.result[chunk_num]
            chunk['hour'] = i
            chunk['chunk'] = chunk_num
            chunk_list.append(chunk)
            
    chunk_list_expanded = []

    for chunk in chunk_list:
        chunk_list_expanded.append(num_words_in_chat(chunk))

    chunk_df = pd.DataFrame(columns = ['created_at', 'updated_at', 'display_name', '_id', 'name', 'type',
           'bio', 'logo', 'body', 'is_action', 'user_badges', 'emoticons', 'hour',
           'chunk', 'num_words_emo', 'num_emo', 'num_words'])

    for chunk in chunk_list_expanded:
        chunk_df = chunk_df.append(chunk)

    chunk_df = chunk_df.astype({
                                'is_action':bool,
                                'hour':int,
                                'chunk':int,
                                'num_words_emo':float,
                                'num_emo':float,
                                'num_words':float
                             })
    return chunk_df, first_stamp

In [24]:
chunk_df, first_stamp = chunker(big_df, min_=2)

/home/jupyter-pomkos/.conda/envs/data_env/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jupyter-pomkos/.conda/envs/data_env/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jupyter-pomkos/.conda/envs/data_env/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

## Save Results

In [26]:
# Format results
for col in ['num_words_emo','num_emo','num_words_only']:
    results = results_formatter(chunk_df, goal=col)
    json_results = d.results_jsonified(results, first_stamp, results_col = col)
    d.save_json(json_results,name=f'algo3.5_{col}')

Saved to data/algo3.5_num_words_emo.json
Saved to data/algo3.5_num_emo.json
Saved to data/algo3.5_num_words_only.json


## Number of unique words

# EDA

In [ ]:
df_counts = chunk_df.groupby(['hour','chunk'])['num_emo','num_words_only'].sum().reset_index()

## Words + Emoji

In [ ]:
# results df
results.head()

In [ ]:
# chunk_df df
chunk_words_sum = chunk_df.groupby(['hour','chunk'])['num_words_emo'].sum().reset_index()

In [ ]:
g = sns.FacetGrid(col='hour',data=chunk_words_sum, hue='hour', col_wrap = 4)
g.map(sns.barplot, 'chunk','num_words_emo')

## Emoji

In [ ]:
gr.plot_with_time(x='created_at', y='num_emo', data = chunk_df)

In [ ]:
sns.catplot(x='hour',y='num_emo',data=df_counts, kind='bar')
plt.title("Mean number of emoticons each chunk per hour")

In [ ]:
g = sns.FacetGrid(col='hour',data=df_counts, hue='hour', col_wrap = 4)
g.map(sns.barplot, 'chunk','num_emo')
plt.suptitle("Number of emoticons per chunk each hour",y=1.05)

## Words

In [ ]:
gr.plot_with_time(x='created_at', y='num_words_only', data = chunk_df)

In [ ]:
sns.catplot(x='hour',y='num_words_only',data=df_counts, kind='bar')
plt.title("Mean number of words each chunk per hour")

In [ ]:
g = sns.FacetGrid(col='hour',data=df_counts, hue='hour', col_wrap = 4)
g.map(sns.barplot, 'chunk','num_words_only')